# Spark SQL

Uses SparkSession instead of SparkContext

## DataFrames:
* Contain rows.
* Can run SQL queries.
* Read and write JSON, Hive, parquet.
* Communicate with JDBC/ODBC, Tableau.
* Schema in inferred at runtime.

## DataSets:
* Can wrap a given struct or type (know up front what is inside it).
* Schema can be inferred at compile time.
* More efficient.

In [1]:
import org.apache.spark.sql._

// Define a schema for our data
case class Person(id:Int, name:String, age:Int, friends:Int)

Intitializing Scala interpreter ...

Spark Web UI available at http://315284ee4037:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1625095554604)
SparkSession available as 'spark'


import org.apache.spark.sql._
defined class Person


In [2]:
// Example using SQL

val schemaPeople = spark.read
.option("header", "true")
.option("inferSchema","true")
.csv("data/fakefriends.csv")
.as[Person] //Takes the DataFrame and converts into a DataSet

schemaPeople.printSchema()

// Creates a temporary database that can be queried
schemaPeople.createOrReplaceTempView("people")

val teenagers = spark.sql("SELECT * FROM people WHERE age <=19")

val results = teenagers.collect()

results.foreach(println)

//spark.stop()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)

[21,Miles,19,268]
[52,Beverly,19,269]
[54,Brunt,19,5]
[106,Beverly,18,499]
[115,Dukat,18,397]
[133,Quark,19,265]
[136,Will,19,335]
[225,Elim,19,106]
[304,Will,19,404]
[341,Data,18,326]
[366,Keiko,19,119]
[373,Quark,19,272]
[377,Beverly,18,418]
[404,Kasidy,18,24]
[409,Nog,19,267]
[439,Data,18,417]
[444,Keiko,18,472]
[492,Dukat,19,36]
[494,Kasidy,18,194]


schemaPeople: org.apache.spark.sql.Dataset[Person] = [id: int, name: string ... 2 more fields]
teenagers: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]
results: Array[org.apache.spark.sql.Row] = Array([21,Miles,19,268], [52,Beverly,19,269], [54,Brunt,19,5], [106,Beverly,18,499], [115,Dukat,18,397], [133,Quark,19,265], [136,Will,19,335], [225,Elim,19,106], [304,Will,19,404], [341,Data,18,326], [366,Keiko,19,119], [373,Quark,19,272], [377,Beverly,18,418], [404,Kasidy,18,24], [409,Nog,19,267], [439,Data,18,417], [444,Keiko,18,472], [492,Dukat,19,36], [494,Kasidy,18,194])


In [13]:
// Example using DataSet
val people = schemaPeople

people.printSchema()

people.select("name").show(5)

people.filter(people("age") < 21).show(5)

people.groupBy(people("age")).count().show(5)

people.withColumn("newColumn", people("age") + 10).show(5)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)

+--------+
|    name|
+--------+
|    Will|
|Jean-Luc|
|    Hugh|
|  Deanna|
|   Quark|
+--------+
only showing top 5 rows

+---+-------+---+-------+
| id|   name|age|friends|
+---+-------+---+-------+
| 21|  Miles| 19|    268|
| 48|    Nog| 20|      1|
| 52|Beverly| 19|    269|
| 54|  Brunt| 19|      5|
| 60| Geordi| 20|    100|
+---+-------+---+-------+
only showing top 5 rows

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
| 34|    6|
| 28|   10|
+---+-----+
only showing top 5 rows

+---+--------+---+-------+---------+
| id|    name|age|friends|newColumn|
+---+--------+---+-------+---------+
|  0|    Will| 33|    385|       43|
|  1|Jean-Luc| 26|      2|       36|
|  2|    Hugh| 55|    221|       65|
|  3|  Deanna| 40|    465|       50|
|  4|   Quark| 68|     21|       78|
+---+--------+---+-------+---------

people: org.apache.spark.sql.Dataset[Person] = [id: int, name: string ... 2 more fields]


In [23]:
// Average friends by age

val people = spark.read
.option("header", "true")
.option("inferSchema", "true")
.csv("data/fakefriends.csv")

people.printSchema()

people.select("age", "friends").groupBy("age").avg("friends").show(5)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)

+---+------------------+
|age|      avg(friends)|
+---+------------------+
| 31|            267.25|
| 65|             298.2|
| 53|222.85714285714286|
| 34|             245.5|
| 28|             209.1|
+---+------------------+
only showing top 5 rows



people: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


In [24]:
// Word counter example
case class Book(value:String)

defined class Book


In [31]:
val input = spark.read.text("data/book.txt").as[Book]

val words = input
.select(explode(split($"value", "\\W+")).alias("word"))
.filter($"word" =!= "")

val lowerCaseWords = words.select(lower($"word").alias("word"))

val wordCountr = lowerCaseWords.groupBy("word").count()

val wordCountrSorted = wordCountr.sort(desc("count"))

wordCountrSorted.show(5) // show(wordCountrSorted.count.toInt)

+----+-----+
|word|count|
+----+-----+
| you| 1878|
|  to| 1828|
|your| 1420|
| the| 1292|
|   a| 1191|
+----+-----+
only showing top 5 rows



input: org.apache.spark.sql.Dataset[Book] = [value: string]
words: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [word: string]
lowerCaseWords: org.apache.spark.sql.DataFrame = [word: string]
wordCountr: org.apache.spark.sql.DataFrame = [word: string, count: bigint]
wordCountrSorted: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [word: string, count: bigint]


In [35]:
// Making the example above easier using RDD + DS
val rdd = sc.textFile("data/book.txt")

val ds = rdd.flatMap(x => x.split("\\W+")).map(x => x.toLowerCase()).toDS()
ds.groupBy("value").count().sort(desc("count")).show(5)

+-----+-----+
|value|count|
+-----+-----+
|  you| 1878|
|   to| 1828|
| your| 1420|
|  the| 1292|
|    a| 1191|
+-----+-----+
only showing top 5 rows



rdd: org.apache.spark.rdd.RDD[String] = data/book.txt MapPartitionsRDD[246] at textFile at <console>:33
ds: org.apache.spark.sql.Dataset[String] = [value: string]


In [38]:
// Min temperature example
import org.apache.spark.sql.types.{FloatType, IntegerType, StringType, StructType}
import org.apache.spark.sql.functions._

case class Temperature(stationID: String, date: Int, measure_type: String, temperature: Float)

import org.apache.spark.sql.types.{FloatType, IntegerType, StringType, StructType}
import org.apache.spark.sql.functions._
defined class Temperature


In [58]:
// Specify the schema (instead of infer)
val temperatureSchema = new StructType()
.add("stationID", StringType, nullable = true)
.add("date", IntegerType, nullable = true)
.add("measure_type", StringType, nullable = true)
.add("temperature", FloatType, nullable = true)

val ds = spark.read
.schema(temperatureSchema)
.csv("data/1800.csv")
.as[Temperature]

ds.filter($"measure_type" === "TMIN")
.select("stationID", "temperature")
.groupBy("stationID").min("temperature")
.withColumn("C", $"min(temperature)" / 10)
.withColumn("F", round($"C" * 9 / 5 + 32, 2))
.show(5)

+-----------+----------------+-----+----+
|  stationID|min(temperature)|    C|   F|
+-----------+----------------+-----+----+
|ITE00100554|          -148.0|-14.8|5.36|
|EZE00100082|          -135.0|-13.5| 7.7|
+-----------+----------------+-----+----+



temperatureSchema: org.apache.spark.sql.types.StructType = StructType(StructField(stationID,StringType,true), StructField(date,IntegerType,true), StructField(measure_type,StringType,true), StructField(temperature,FloatType,true))
ds: org.apache.spark.sql.Dataset[Temperature] = [stationID: string, date: int ... 2 more fields]


In [70]:
// Total spent by customer

val ds = spark.read
.schema(
    new StructType()
    .add("customerID", IntegerType)
    .add("itemID", IntegerType)
    .add("price", FloatType)
)
.csv("data/customer-orders.csv")

val results = ds
.select("customerID", "price")
.groupBy("customerID").agg(round(sum("price"),2).alias("TotalSpent"))
.sort(desc("TotalSpent"))

results.show(5)

+----------+----------+
|customerID|TotalSpent|
+----------+----------+
|        68|   6375.45|
|        73|    6206.2|
|        39|   6193.11|
|        54|   6065.39|
|        71|   5995.66|
+----------+----------+
only showing top 5 rows



ds: org.apache.spark.sql.DataFrame = [customerID: int, itemID: int ... 1 more field]
results: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [customerID: int, TotalSpent: double]
